# Demo of overlay of comparison stars

## Go to "Cell → Run all" then scroll to the bottom of the page

### Be patient while the image is downloaded

In [ ]:
import pandas
from astropy.table import Table
from astropy.coordinates import SkyCoord, match_coordinates_sky
from astropy import units as u
from astropy.nddata import CCDData
from pathlib import Path
from glowing_waffles.differential_photometry import *
from astrowidgets import ImageWidget, __version__
from glowing_waffles.photometry import *
import numpy as np
from comparison_functions import *
import astrowidgets
from seeing_profile_functions import *
import ipywidgets as ipw

In [ ]:
print(astrowidgets.__version__)

In [ ]:
directory_with_images = '.'
#This could be any image from roughly the middle of the sequence.
sample_image_for_finding_stars = 'https://zenodo.org/record/3356833/files/kelt-16-b-S001-R001-C084-r.fit.bz2?download=1'
object_of_interest = 'kelt-16b'

In [ ]:
df, RD, ra, dec, RD['coords'] = read_file('gaia_stars_1571495728.radec')

In [ ]:
ccd, ccdx, ccdy, vsx, ra, dec, vsx['coords'], vsx_names = set_up('kelt-16b', directory_with_images, sample_image_for_finding_stars)

In [ ]:
apass, v_angle, RD_angle = match(ccd, RD, vsx)

In [ ]:
Cmag = 11.380999999999998

In [ ]:
apass_good_coord, good_stars = mag_scale(Cmag, apass, v_angle, RD_angle)

In [ ]:
ent = in_field(apass_good_coord, ccd, ccdx, ccdy, apass, good_stars)

In [ ]:
iw = ImageWidget()
out = ipw.Output()
bind_map = iw._viewer.get_bindmap()
gvc = iw._viewer.get_canvas()
bind_map.map_event(None, ('shift',), 'ms_left', 'cursor')
gvc.add_callback('cursor-down', wrap(iw, out))

In [ ]:
header = ipw.HTML(value="""
<h2>Scroll to pan, use +/- keys to zoom</h2>
<h3>Left click to exclude star as target or comp. Click again to include.</h3>
""")

legend = ipw.HTML(value="""
<h3>Green circles -- Gaia stars within 2.5 arcmin of target</h3>
<h3>Blue circles -- APASS stars within 1 mag of target</h3>
<h3>Red circles -- VSX variables</h3>
<h3>Red × -- Exclude as target or comp</h3>
""")

In [ ]:
# make_markers needs a local file for now...
ccd.write('image_file.fits', overwrite=True)
make_markers(iw, directory_with_images, 'image_file.fits', RD, vsx, ent, 'kelt-16b')

In [ ]:
box = ipw.VBox()
inner_box = ipw.HBox()
inner_box.children = [iw, legend]
box.children = [header, inner_box]
box